### This notebook is to generate df of Positions, that is resulted in our strategy

In [28]:
import pandas as pd
from math import sqrt
import math

### Functions

In [74]:
# L_you calculation / inRange
# 4B. JNP: Use 'get_liquidity' function to calculate liquidity as a function of amounts and price range
def get_liquidity0(sqrtA,sqrtB,amount0,decimals):
    
    if (sqrtA > sqrtB):
          (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount0/((2**96*(sqrtB-sqrtA)/sqrtB/sqrtA)/10**decimals)
    return liquidity

def get_liquidity1(sqrtA,sqrtB,amount1,decimals):
    
    if (sqrtA > sqrtB):
        (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
    liquidity=amount1/((sqrtB-sqrtA)/2**96/10**decimals)
    return liquidity

def get_liquidity(tick,tickA,tickB,amount0,amount1,decimal0,decimal1):
    
        sqrt=(1.0001**(tick/2)*(2**96))
        sqrtA=(1.0001**(tickA/2)*(2**96))
        sqrtB=(1.0001**(tickB/2)*(2**96))
        if (sqrtA > sqrtB):
            (sqrtA,sqrtB)=(sqrtB,sqrtA)
    
        if sqrt<=sqrtA:
            
            liquidity0=get_liquidity0(sqrtA,sqrtB,amount0,decimal0)
            inRange = False
            return liquidity0, inRange
        elif sqrt<sqrtB and sqrt>sqrtA:

            liquidity0=get_liquidity0(sqrt,sqrtB,amount0,decimal0)
            liquidity1=get_liquidity1(sqrtA,sqrt,amount1,decimal1)
            
            liquidity=liquidity0 if liquidity0<liquidity1 else liquidity1
            inRange = True
            return liquidity, inRange
        
        else:
            liquidity1=get_liquidity1(sqrtA,sqrtB,amount1,decimal1)
            inRange = False
            return liquidity1, inRange

In [85]:
def price_to_tick(price_):
    # convert price (token1 by token0) to tick
    tick = math.floor(math.log(sqrt(price_), math.sqrt(1.0001)))
    return tick

### Main

In [113]:
### input: money USD, upper and lower
# maths:
# p(i) = 1.0001**i , where i = tick
# i = math.floor(math.log(sqrt(P)/(2**96), math.sqrt(1.0001)))

# of positions:
# liquidity L = sqrt(token0_amt*token1__amt) ?? > may need the complicated 3 cases maths
# - Fee income = (L_you/L_pool) * swap volume under fixed time period (USD) * pool_fee_rate/100

In [114]:
# 0. PROVIDE BY USERS
# INVEST_AMOUNT = 100000 # in USD
UPPER = 1/2197.405# 0.00050567 # 0.00051486 # token1 per token0 (in terms of price)
LOWER = 1/2376.6075 # 0.00038603 # 0.00039069
FEETIER = '3000' # 0.3 = 3000/10000
TOKEN0 = 'DAI'
TOKEN1 = 'WETH'
MINT_TIME = 1627437600 # needs to start with the hour (unixTime % 3600 = 0) 
# Wed Jul 28 2021 09:00:00 GMT+0700 (+07)

In [115]:
# 1. GET BY QUERYING, at specific mint time (TODO)
CPRICE = 1/2293.4514 # 0.000436882 # ETH per DAI

# query Token from the graph, or Etherscan for now. (TODO)
DECIMAL0 = 18 # DAI
DECIMAL1 = 18 # WETH

In [116]:
# 2. CALCULATED BY ALGORITHM (TODO, or just manual for now)
amt0 = 5457.48  # 531910.2
amt1 = 1.98  # 199

In [117]:
# 3. GET from DF_MERGE (for specific hour frame)
L_POOL = 2257382392731291728666040# DF_MERGE
SWAP_VOLUME = 671187.484515 # of that hour, in USD (?) - is this sqrt(P)

In [118]:
tick = price_to_tick(CPRICE)
tickA = price_to_tick(LOWER)
tickB = price_to_tick(UPPER)
L_you, inRange = get_liquidity(tick,tickA,tickB,amt0,amt1,DECIMAL0,DECIMAL1)

In [119]:
if inRange:
    pool_share = (L_you/L_POOL)
    pool_fee_rate = float(FEETIER) / 10000
    fees = pool_share * SWAP_VOLUME * pool_fee_rate / 100


In [120]:
inRange
fees

4.794602974269194

In [56]:
# Fees calculation
# Fee income = (L_you/L_pool) * swap volume under fixed time period * pool_fee_rate/100

L_pool = 2257382392731291728666040 # 7732247
swapVolume = 671187.484515

pool_fee_rate = float(FEETIER) / 10000

fees = (L_you/L_pool) * swapVolume * pool_fee_rate / 100

In [57]:
fees

78.54793966842925

In [55]:
# # 4A. Flipside: Calculate L_you and check if we are in range

# if CPRICE <= LOWER:
#     L_you = amt0 * (sqrt(UPPER) * sqrt(LOWER)) / (sqrt(UPPER) - sqrt(LOWER))
#     # shouldnt this be 0 ? have liquidity but no fees?
#     inRange = False
# elif CPRICE > UPPER:
#     L_you = amt1 / (sqrt(UPPER) - sqrt(LOWER))
#     inRange = False
#     # shouldnt this be 0 ?
# else:
#     liq1 = amt0 * (sqrt(UPPER) * sqrt(CPRICE)) / (sqrt(UPPER) - sqrt(CPRICE))
#     liq2 = amt1 / (sqrt(CPRICE) - sqrt(LOWER))
#     L_you = min(liq1,liq2)
#     inRange = True

# print(L_you)
# print(inRange)


15150.593942102827
True


In [50]:
df_merged = pd.read_csv('../data/df_merged_DAI-WETH-3000-timestamp-1627466400-1620158400.csv')
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data, note: amountUSD is sqrt(P)
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour (liquidity is for the whole pool)
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
# https://github.com/Uniswap/uniswap-v3-subgraph/blob/main/schema.graphql
df_merged[watch_list]

periodStartUnix  txCount  swaps_txCount        amount0       amount1  \
0          1627466400        6            NaN            NaN           NaN   
1          1627462800       13           11.0  299616.218709 -1.304435e+02   
2          1627459200       23           16.0 -105151.549893  4.687934e+01   
3          1627455600       31           27.0  753317.520533 -3.305345e+02   
4          1627452000       17           17.0 -879730.423181  3.878647e+02   
...               ...      ...            ...            ...           ...   
2013       1620172800        2            1.0    -331.043259  1.000000e-01   
2014       1620169200        1            NaN            NaN           NaN   
2015       1620165600        1            NaN            NaN           NaN   
2016       1620162000        2            1.0      -0.000336  1.000000e-07   
2017       1620158400        6            1.0     -33.854156  1.000000e-02   

          amountUSD   tick                  liquidity  \
0               NaN -77418  2257757512531221050660252   
1     299557.018495 -77395  2258385571433368075249753   
2     671187.484515 -77340  2257382392731291728666040   
3     753167.996064 -77360  2256221622137200723238932   
4     881181.264433 -77220  2261588786047650735673230   
...             ...    ...                        ...   
2013       0.000000 -80932      379427976363351507096   
2014            NaN -81234      379427976363351507096   
2015            NaN -81234      232295550235364197007   
2016       0.000000 -81234      229713263233648638236   
2017       0.000000 -81234      222633640125805970242   

                         sqrtPrice        tvlUSD pool.token0.symbol  \
0     1651413884466706248548127541  2.834231e+07                DAI   
1     1653332807396390538189673451  2.828409e+07                DAI   
2     1657909038401687840269395027  2.808664e+07                DAI   
3     1656280685119642923297705596  2.809426e+07                DAI   
4     1667896272296713929624373234  2.782114e+07                DAI   
...                            ...           ...                ...   
2013  1385391852998664492209230571  0.000000e+00                DAI   
2014  1364573546772589608591144427  0.000000e+00                DAI   
2015  1364573546772589608591144427  0.000000e+00                DAI   
2016  1364573546772589608591144427  0.000000e+00                DAI   
2017  1364573512386034424627810688  0.000000e+00                DAI   

     pool.token1.symbol  token0Price  token1Price  
0                  WETH  2301.692508     0.000434  
1                  WETH  2296.352740     0.000435  
2                  WETH  2283.693255     0.000438  
3                  WETH  2288.185835     0.000437  
4                  WETH  2256.425980     0.000443  
...                 ...          ...          ...  
2013               WETH  3270.498157     0.000306  
2014               WETH  3371.050602     0.000297  
2015               WETH  3371.050602     0.000297  
2016               WETH  3371.050602     0.000297  
2017               WETH  3371.050772     0.000297  

[2018 rows x 14 columns]

In [18]:
math.floor(math.log(1653332807396390538189673451/(2**96), math.sqrt(1.0001)))

-77395

In [60]:
2257757512531221050660252
1430725709192736851216600
44406878979210543
2272735397855406364299775

2257757512531221050660252

In [58]:
2257757512531221050660252/2**96

2.8496906161678703e-05

In [ ]:
sitions": [
      {
        "liquidity": "44406878979210543",
        "tickLower": {
          "tickIdx": "193380"
        },